## Breast Cancer Prediction

In [41]:
# Base libraries
import pandas as pd
import numpy as np

#  Custom helpers
from helpers import ml_tools
from helpers import viz

# Dataset
from sklearn.datasets import load_breast_cancer

# Processing libraries
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve


# Baseline classifier algos
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Utilities
from tempfile import mkdtemp
from shutil import rmtree

# Vizualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio


# Setting options
pd.options.display.max_columns = 25
sns.set(rc={'figure.figsize':(12,6)}, palette='pastel')
seed = 42
viz.change_theme('white')

In [29]:
features, labels = load_breast_cancer(return_X_y=True, as_frame=True)
features.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,...,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,...,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,...,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,...,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,...,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,...,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [30]:
features.shape

(569, 30)

In [31]:
print('Missing values: ', features.isna().sum().sum())

Missing values:  0


In [32]:
corr_heatmap = px.imshow(img=features.corr(),
                         labels=dict(x='', y='', color='Pearson Index'),
                         x=features.corr().columns,
                         y=features.corr().index,
                         color_continuous_scale='GnBu')

corr_heatmap.update_xaxes(side='top')

corr_heatmap.update_layout(
    #title='Correlation heatmap',
    width=1000,
    height=600,
    font=dict(
        family="AkzidGrtskNext-Light",
        size=15),
    coloraxis_colorbar_x=-0.01
)

pio.show(corr_heatmap)

In [33]:
label_cat = labels.map({1:'1', 0:'0'})
count = px.histogram(data_frame=label_cat, 
                     y='target', orientation='h',
                     color='target',
                     color_discrete_sequence=[viz.water, viz.pink],
                     labels={'target': 'Diagnostic', 'count': 'Count'},
                     title='Target repartition'
                    )
count.update_layout(showlegend=False)
pio.show(count)

### Split the dataset

In [44]:
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    labels, 
                                                    test_size=0.2, 
                                                    random_state=seed, 
                                                    stratify=labels)

### Testing various classification baseline algos

In [35]:
scoring_metric = 'recall'

base_models = list()
base_models.extend([
    ('Logistic Regression', LogisticRegression()),
    ('Linear Discriminant Analisys', LinearDiscriminantAnalysis()),
    ('Kneighbors', KNeighborsClassifier()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Naive Bayes', GaussianNB()),
    ('SVM Linear', SVC(kernel='linear')),
    ('SVM Radial', SVC(kernel='rbf'))
])

baseline_summary = pd.DataFrame(data=None, columns=['Name', 'Model', 'Recall Mean', 'Recall Std'])

results = list()
names = list()

for idx, (name, model) in enumerate(base_models):
    folds = KFold(n_splits=10, shuffle=True, random_state=seed)
    p = Pipeline([
        ('scaler', StandardScaler()),
        ('PCA', PCA(n_components=4)),
        ('algo', model)
    ])

    recall = cross_val_score(estimator=p, 
                             X=X_train,
                             y=y_train, 
                             scoring=scoring_metric, 
                             cv=folds,
                             n_jobs=2)
    
    baseline_summary.loc[idx] = [name, model, round(recall.mean(), 3), round(recall.std(), 3)]
    baseline_summary = baseline_summary.sort_values('Recall Mean', ascending=False).reset_index(drop=True)
    results.append(recall)
    names.append(name)

#### Summary

In [36]:
baseline_summary

,Name,Model,Recall Mean,Recall Std
0,Linear Discriminant Analisys,LinearDiscriminantAnalysis(),0.990,0.016
1,Logistic Regression,LogisticRegression(),0.983,0.017
2,SVM Linear,SVC(kernel='linear'),0.979,0.024
3,Kneighbors,KNeighborsClassifier(),0.975,0.028
4,SVM Radial,SVC(),0.974,0.025
5,Naive Bayes,GaussianNB(),0.951,0.045
6,Decision Tree,DecisionTreeClassifier(),0.932,0.040


#### Visualize the results

In [37]:
fig_data = pd.DataFrame(np.array(results).transpose(), columns=names)
fig = px.box(data_frame=fig_data, 
             labels={'variable':'Models', 'value':'Recall over 10 folds'}, 
             title='Models efficiency', color_discrete_sequence=[viz.blue])
fig.update_xaxes(categoryorder='mean descending')
pio.show(fig)

### Cross validation with the best candidate
Linear Discriminant Analysis with PCA with 4 components (kinda problematic since the algo is already a linear algo based on dimensionality reduction, more on that later)

#### Evaluating all main classification metrics on the best model

In [38]:
test_model = Pipeline([
        ('scaler', StandardScaler()),
        ('PCA', PCA(n_components=4)),
        ('algo', LinearDiscriminantAnalysis())
    ])

ml_tools.cross_val_metrics(test_model, X_train, y_train)


[Accuracy] : 0.94493 (+/- 0.03744)
[Precision] : 0.92628 (+/- 0.05975)
[Recall] : 0.98951 (+/- 0.01617)
[F1] : 0.95572 (+/- 0.03193)


In [39]:
# testing with PCA
test_model = Pipeline([
        ('scaler', StandardScaler()),
        ('algo', LinearDiscriminantAnalysis())
    ])

ml_tools.cross_val_metrics(test_model, X_train, y_train)

[Accuracy] : 0.95594 (+/- 0.02630)
[Precision] : 0.93981 (+/- 0.03544)
[Recall] : 0.99321 (+/- 0.01372)
[F1] : 0.96545 (+/- 0.02129)


### Hypertuning the LDA model

In [ ]:
print([x.round(2) for x in np.arange (0.1, 1, 0.2)])

#### Testing the model

In [12]:
lda_baseline = Pipeline([
    ('scaler', StandardScaler()),
    ('PCA', PCA(n_components=4)),
    ('algo', LinearDiscriminantAnalysis())
    ])

lda_baseline.fit(X_train, y_train)
y_pred = lda_baseline.predict(X_test)
y_score = lda_baseline.predict_proba(X_test)[:,1]

viz.classification_model_performance_plot('lda_baseline', y_test, y_pred, y_score)

#### Comments
The cumulation of PCA and LDA is not really problematic per se. But since, we can observe a good result with this combination, we can try to ommit the PCA and implement a regularized LDA. For that, Sklearn LDA propose a `shrinkage` parameter in combination with `lsqr` and `eigen` passed as the `solver` argument.

### Testing the Linear Discriminent Analysis without PCA

In [15]:
cachedir = mkdtemp()
folds = KFold(n_splits=10, shuffle=True, random_state=seed)

lda_only = Pipeline([
    ('scaler', StandardScaler()),
    ('lda', LinearDiscriminantAnalysis())
    ])

params= [{
    'lda__solver':['svd']
},
    {'lda__solver':['lsqr', 'eigen'], 
     'lda__shrinkage':['auto', None, *np.arange (0.1, 1, 0.2)], 
}]

grid_lda_only = GridSearchCV(estimator=lda_only, 
                    param_grid=params,
                    scoring='recall',
                    cv=folds, 
                    refit=True,
                    return_train_score=True, 
                    verbose=0)



scores = grid_lda_only.fit(X_train, y_train)
print('DONE')
rmtree(cachedir)

DONE


#### Evaluating all main classification metrics on the best model

In [16]:
lda_only_model = Pipeline([
        ('scaler', StandardScaler()),
        ('lda', LinearDiscriminantAnalysis())
    ])
best_params = grid_lda_only.best_params_
lda_only_model.set_params(**best_params)
ml_tools.cross_val_metrics(lda_only_model, X_train, y_train)

[Accuracy] : 0.95816 (+/- 0.03748)
[Precision] : 0.93734 (+/- 0.05581)
[Recall] : 1.00000 (+/- 0.00000)
[F1] : 0.96679 (+/- 0.03009)


#### Testing the model

In [17]:
lda_only_model.fit(X_train, y_train)
y_pred = lda_only_model.predict(X_test)
y_score = lda_only_model.predict_proba(X_test)[:,1]

viz.classification_model_performance_plot('lda_only_model', y_test, y_pred, y_score)

In [18]:
print(lda_baseline)
print(lda_only_model)

Pipeline(steps=[('scaler', StandardScaler()), ('PCA', PCA(n_components=4)),
                ('algo', LinearDiscriminantAnalysis())])
Pipeline(steps=[('scaler', StandardScaler()),
                ('lda',
                 LinearDiscriminantAnalysis(shrinkage=0.5000000000000001,
                                            solver='lsqr'))])


### Trying to optimize a Support Vector Machine
#### Model Baseline

In [19]:
# temp directory
cachedir = mkdtemp()

model = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('SupVM', SVC())
],
    memory=cachedir)

### Tuning with GridSearchCV

In [20]:
folds = KFold(n_splits=10, shuffle=True, random_state=seed)

params= [{
         'pca__n_components': [2,3,4], 
         'SupVM__kernel':['rbf'],
         'SupVM__C': [0.1, 0.5, 1, 10,30, 40, 50, 75, 100, 500, 1000], 
         'SupVM__gamma' : [0.01, 0.05, 0.07, 0.1, 0.5, 1, 5, 10, 50]
},
         {
         'pca__n_components': [2,3,4], 
         'SupVM__kernel':['linear'],
         'SupVM__C': [0.1, 0.5, 1, 10,30, 40, 50, 75, 100, 500, 1000]
}]

grid = GridSearchCV(estimator=model, 
                    param_grid=params,
                    scoring='recall',
                    cv=folds, 
                    refit=True,
                    return_train_score=True, 
                    verbose=0)
scores = grid.fit(X_train, y_train)

params_r2 = [{
         'pca__n_components': [4, 5], 
         'SupVM__kernel':['rbf'],
         'SupVM__C': [0.001, 0.01, 0.1], 
         'SupVM__gamma' : [0.5]
}]

grid_2 = GridSearchCV(estimator=model, 
                    param_grid=params_r2,
                    scoring='recall',
                    cv=folds, 
                    refit=True,
                    return_train_score=True, 
                    verbose=0)

scores_2 = grid_2.fit(X_train, y_train)

print('DONE')
rmtree(cachedir)

DONE


### Test the model
#### Round 1 of hypertuning

In [21]:
# obtain best parameters
grid.best_params_
print('best parameters: \n'.upper(), *grid.best_params_.items(), sep='\n', end='\n\n')

# obtain the best score (mean of the scoring method over the different splits)
grid.best_score_
print('best mean recall: \n'.upper(), grid.best_score_, '(note: it\'s not the generalized score)', end='\n\n')

# obtain the generalized score
s = grid.score(X_test, y_test)
print('generalized score: \n'.upper(), s, end='\n\n')

# obtain the best model — which contain features importance and coefs 
# *note* feature importance absent for SVM (dimension changes) and coefs only available if kernel=linear
grid.best_estimator_
print('best model: \n'.upper(), grid.best_estimator_, end='\n\n')

BEST PARAMETERS: 

('SupVM__C', 0.1)
('SupVM__gamma', 0.5)
('SupVM__kernel', 'rbf')
('pca__n_components', 4)

BEST MEAN RECALL: 
 1.0 (note: it's not the generalized score)

GENERALIZED SCORE: 
 1.0

BEST MODEL: 
 Pipeline(memory='/var/folders/32/mb3r84313qjb1n42wc7ck7c80000gn/T/tmpe097l7t9',
         steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=4)),
                ('SupVM', SVC(C=0.1, gamma=0.5))])



In [22]:
# Reproduce and fit the model
# note : we must add SVM__probability=True since SVM dont return proba by default

round_1_params = grid.best_params_ | dict(SupVM__probability=True)
final_SVM_model = model.set_params(**round_1_params)
fitted_final_SVM_model = final_SVM_model.fit(X_train, y_train);

In [23]:
# Complete overview of the results

y_pred = fitted_final_SVM_model.predict(X_test)
y_score = fitted_final_SVM_model.predict_proba(X_test)[:,1]

viz.classification_model_performance_plot('fitted_final_SVM_model', y_test, y_pred, y_score)

#### Round 2 of hypertuning

In [24]:
# obtain best parameters
grid_2.best_params_
print('best parameters: \n'.upper(), *grid_2.best_params_.items(), sep='\n', end='\n\n')

# obtain the best score (mean of the scoring method over the different splits)
grid_2.best_score_
print('best mean recall: \n'.upper(), grid_2.best_score_, '(note: it\'s not the generalized score)', end='\n\n')

# obtain the generalized score
s = grid_2.score(X_test, y_test)
print('generalized score: \n'.upper(), s, end='\n\n')

# obtain the best model — which contain features importance and coefs 
# *note* feature importance absent for SVM (dimension changes) and coefs only available if kernel=linear
grid_2.best_estimator_
print('best model: \n'.upper(), grid_2.best_estimator_, end='\n\n')

BEST PARAMETERS: 

('SupVM__C', 0.001)
('SupVM__gamma', 0.5)
('SupVM__kernel', 'rbf')
('pca__n_components', 4)

BEST MEAN RECALL: 
 1.0 (note: it's not the generalized score)

GENERALIZED SCORE: 
 1.0

BEST MODEL: 
 Pipeline(memory='/var/folders/32/mb3r84313qjb1n42wc7ck7c80000gn/T/tmpe097l7t9',
         steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=4)),
                ('SupVM', SVC(C=0.001, gamma=0.5))])



In [25]:
# Reproduce and fit the model
# note : we must add SVM__probability=True since SVM dont return proba by default

round_2_params = grid_2.best_params_ | dict(SupVM__probability=True)
final_SVM_model_2 = model.set_params(**round_2_params)
fitted_final_SVM_model_2 = final_SVM_model_2.fit(X_train, y_train);

In [26]:
# Complete overview of the results

y_pred = fitted_final_SVM_model_2.predict(X_test)
y_score = fitted_final_SVM_model_2.predict_proba(X_test)[:,1]

viz.classification_model_performance_plot('fitted_final_SVM_model', y_test, y_pred, y_score)

In [27]:
print(fitted_final_SVM_model)
print(fitted_final_SVM_model_2)

Pipeline(memory='/var/folders/32/mb3r84313qjb1n42wc7ck7c80000gn/T/tmpe097l7t9',
         steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=4)),
                ('SupVM', SVC(C=0.001, gamma=0.5, probability=True))])
Pipeline(memory='/var/folders/32/mb3r84313qjb1n42wc7ck7c80000gn/T/tmpe097l7t9',
         steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=4)),
                ('SupVM', SVC(C=0.001, gamma=0.5, probability=True))])
